<a href="https://colab.research.google.com/github/mntalha/Autoencoder/blob/main/TestCdVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

Author: Kamal Choudhary (kamal.choudhary@nist.gov)

In [1]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install -q pytorch-lightning==1.3.6

In [2]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 253 (delta 38), reused 52 (delta 32), pack-reused 164
Receiving objects: 100% (253/253), 146.69 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Updating files: 100% (99/99), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [3]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [4]:
!echo $HYDRA_FULL_ERROR

1


In [5]:
!echo "PROJECT_ROOT=/content/cdvae" > .env
!echo "HYDRA_JOBS=/content/cdvae/HYDRA_JOBS" >> .env
!echo "WABDB_DIR=/content/cdvae/WABDB" >> .env
!cat .env

PROJECT_ROOT=/content/cdvae
HYDRA_JOBS=/content/cdvae/HYDRA_JOBS
WABDB_DIR=/content/cdvae/WABDB


Data was generated using this [script](https://github.com/JARVIS-Materials-Design/cdvae/blob/main/scripts/generate_data_cdvae.py).

In [22]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/data/carbon.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['datamodule']['batch_size']['train']= 16
yam['datamodule']['batch_size']['val']= 16
yam['datamodule']['batch_size']['test']= 16

with open('/content/cdvae/conf/data/carbon.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [26]:
!rm -r HYDRA_JOBS/*
!rm -r WABDB/*

rm: cannot remove 'HYDRA_JOBS/*': No such file or directory
rm: cannot remove 'WABDB/*': No such file or directory


In [27]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=carbon expname=carbon_test model.predict_property=True

2023-12-12 22:08:59.030981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 22:08:59.031053: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 22:08:59.031093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 22:09:00.556970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), 

In [28]:
!pip install smact

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.0 MB/s eta 0:00:00


In [ ]:
# from datetime import date
# d1 = today.strftime("%Y-%m-%d")

In [29]:
temp_dir=!ls /content/cdvae/HYDRA_JOBS/singlerun/

In [30]:
os.environ['TMP_DIR']=temp_dir[0]

In [31]:
!echo $TMP_DIR

2023-12-12


Adjust path accordingly

In [32]:
%%time
!python scripts/evaluate.py --n_step_each 5 --num_batches_to_samples 5 --batch_size 5 --model_path "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/carbon_test" --tasks opt gen recon

2023-12-12 22:45:01.503903: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 22:45:01.503979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 22:45:01.504016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 22:45:02.862901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT

In [33]:
!pip install matminer==0.8.0
!pip install ruamel.yaml==0.17.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.18.5
    Uninstalling ruamel.yaml-0.18.5:
      Successfully uninstalled ruamel.yaml-0.18.5


In [34]:
%%time
#!python scripts/compute_metrics.py --root_path  "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks   gen recon
!python scripts/compute_metrics.py --root_path  "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/carbon_test" --tasks gen recon

2023-12-12 23:22:12.144186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 23:22:12.144260: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 23:22:12.144301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 23:22:13.778417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT

In [35]:
temp_dir[0]

'2023-12-12'

In [36]:
!ls /content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/carbon_test/

'epoch=154-step=1085.ckpt'   eval_metrics.json	 hparams.yaml	     prop_scaler.pt
 eval_gen.pt		     eval_recon.pt	 lattice_scaler.pt   run.log


In [38]:
import torch
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import pmg_to_atoms
from jarvis.core.lattice import Lattice
from jarvis.core.specie import atomic_numbers_to_symbols
from jarvis.db.jsonutils import dumpjson
from jarvis.analysis.structure.spacegroup import Spacegroup3D
from collections import Counter
from pymatgen.core.structure import Structure
import pandas as pd
opt_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/carbon_test/eval_opt.pt"
gen_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/carbon_test/eval_gen.pt"
recon_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/carbon_test/eval_recon.pt"
csv_path = "/content/cdvae/data/carbon_24/test.csv"
df = pd.read_csv(csv_path)
x = torch.load(recon_path)
y = torch.load(gen_path)
#z = torch.load(opt_path)
#print(len(df),x["num_atoms"].shape,y["num_atoms"].shape,z["num_atoms"].shape)

In [41]:
num_atoms = x["num_atoms"]
atom_types = x["atom_types"]
frac_coords = x["frac_coords"]
lengths = x["lengths"]
angles = x["angles"]
index_list = torch.cumsum(num_atoms[0], dim=0).numpy().tolist()
indice_tuples = []
for i, ii in enumerate(index_list):
    if i == 0:
        tup = [0, index_list[i] - 1]
    else:
        tup = [index_list[i - 1] - 1, index_list[i] - 1]
    indice_tuples.append(tup)

recon_structures = []

for id_needed in range(num_atoms.shape[1]):
    id_fracs = frac_coords[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_atom_types = atom_types[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_lengths = lengths[0].numpy()[id_needed]
    id_angles = angles[0].numpy()[id_needed]
    lat = Lattice.from_parameters(
        id_lengths[0],
        id_lengths[1],
        id_lengths[2],
        id_angles[0],
        id_angles[1],
        id_angles[2],
    ).matrix
    atoms = Atoms(
        lattice_mat=lat,
        elements=atomic_numbers_to_symbols(id_atom_types),
        coords=id_fracs,
        cartesian=False,
    )
    # spg_numb = Spacegroup3D(atoms).space_group_number
    # spg_numbs.append(spg_numb)

    # print()
    # print()
    # print()
    # print("jarvis\n", atoms)
    # struct = Structure(
    #    lattice=Lat.from_parameters(
    #        id_lengths[0],
    #        id_lengths[1],
    #        id_lengths[2],
    #        id_angles[0],
    #        id_angles[1],
    #        id_angles[2],
    #    ),
    #    species=id_atom_types,
    #    coords=id_fracs,
    #    coords_are_cartesian=False,
    # )
    # atoms = pmg_to_atoms(struct)
    # print("pmg\n", atoms)
    # print()
    # print()
    # print()

    # gen_structures.append(atoms.to_dict())
    recon_structures.append(atoms)


In [40]:
test_structures=[]
jids = []
for i,ii in df.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  test_structures.append(atoms)#.to_dict())
  jids.append(ii['material_id'])

In [47]:
df

,Unnamed: 0,material_id,cif,energy_per_atom
0,56,C-13927-8536-14,# generated using pymatgen\ndata_C\n_symmetry_...,-154.546684
1,3900,C-72728-4135-43,# generated using pymatgen\ndata_C\n_symmetry_...,-154.159400
2,863,C-92152-5470-22,# generated using pymatgen\ndata_C\n_symmetry_...,-154.158839
3,4676,C-170380-2255-20,# generated using pymatgen\ndata_C\n_symmetry_...,-154.313358
4,1157,C-193944-7687-47,# generated using pymatgen\ndata_C\n_symmetry_...,-154.305662
...,...,...,...,...
94,3342,C-56503-8782-7,# generated using pymatgen\ndata_C\n_symmetry_...,-154.340324
95,5882,C-102903-5111-47,# generated using pymatgen\ndata_C\n_symmetry_...,-154.146013
96,7215,C-34617-8887-22,# generated using pymatgen\ndata_C\n_symmetry_...,-154.530711
97,2072,C-57148-8436-25,# generated using pymatgen\ndata_C\n_symmetry_...,-154.202201


In [48]:
from jarvis.io.vasp.inputs import Poscar
import json
f=open('AI-AtomGen-energy_per_atom-carbon24-test-rmse.csv','w')
line='id,target,prediction\n'
f.write(line)
for i,j,k in zip(test_structures,recon_structures,jids):
  print(k,i.composition.reduced_formula,j.composition.reduced_formula)
  line = k+","+Poscar(i).to_string().replace('\n','\\n')+","+Poscar(j).to_string().replace('\n','\\n')+"\n"
  #line = k+","+json.dumps(i.to_dict())+","+json.dumps(j.to_dict())+"\n"
  f.write(line)
f.close()
#zip file before uploading to JARVIS-Leadrboard

C-13927-8536-14 C C
C-72728-4135-43 C C
C-92152-5470-22 C C
C-170380-2255-20 C C
C-193944-7687-47 C C
C-145323-1843-37 C C
C-34623-4-17 C C
C-73665-9416-19 C C
C-134173-4385-29 C C
C-170378-1288-38 C C
C-96676-423-51 C C
C-176687-5509-13 C C
C-157685-398-45 C C
C-176654-3153-46 C C
C-136247-3248-35 C C
C-53804-8031-29 C C
C-96669-7803-47 C C
C-47644-8979-54 C C
C-134208-315-24 C C
C-157668-5131-20 C C
C-47620-4862-49 C C
C-176673-628-15 C C
C-96698-840-45 C C
C-104299-1094-60 C C
C-148275-4529-35 C C
C-34660-1476-28 C C
C-184035-5891-45 C C
C-28266-8419-4 C C
C-176683-1873-36 C C
C-172965-3737-15 C C
C-13927-8536-35 C C
C-41306-4542-24 C C
C-170362-9529-41 C C
C-56518-9542-30 C C
C-141026-3786-37 C C
C-73651-4102-35 C C
C-40118-1783-42 C C
C-142789-7601-10 C C
C-104307-940-37 C C
C-53824-8786-8 C C
C-141022-7340-3 C C
C-53814-5771-49 C C
C-80155-5756-2 C C
C-56491-7685-5 C C
C-136225-3934-1 C C
C-145327-8310-9 C C
C-142748-3187-22 C C
C-130238-8833-50 C C
C-57113-4466-43 C C
C-157707-3

In [50]:
dfx = pd.read_csv('AI-AtomGen-energy_per_atom-carbon24-test-rmse.csv')
dfx

,id,target,prediction
0,C-13927-8536-14,System\n1.0\n2.424583551895228 0.0 0.556862320...,System\n1.0\n2.54003 0.0 -0.68426\n-0.46907 3....
1,C-72728-4135-43,System\n1.0\n2.409696672631902 0.0 -0.50076834...,System\n1.0\n2.47226 0.0 -0.09561\n-0.9939 3.1...
2,C-92152-5470-22,System\n1.0\n2.0683286251813 0.0 1.37221938302...,System\n1.0\n2.69893 0.0 -0.12832\n0.29249 4.4...
3,C-170380-2255-20,System\n1.0\n2.3986593887891314 0.0 -0.6356704...,System\n1.0\n2.42285 0.0 -0.14772\n0.07088 3.5...
4,C-193944-7687-47,System\n1.0\n3.3574633614171208 0.0 0.52349561...,System\n1.0\n2.65839 0.0 -0.2664\n-0.09847 3.4...
...,...,...,...
94,C-56503-8782-7,System\n1.0\n2.3018678003472304 0.0 -0.9541864...,System\n1.0\n2.35798 0.0 0.14226\n0.2979 4.326...
95,C-102903-5111-47,System\n1.0\n4.197969915441017 0.0 -0.00084258...,System\n1.0\n3.24563 0.0 -0.03253\n0.20548 3.7...
96,C-34617-8887-22,System\n1.0\n2.480119973282893 0.0 0.000364037...,System\n1.0\n2.42683 0.0 -0.018\n-0.02658 2.42...
97,C-57148-8436-25,System\n1.0\n2.4211418384013887 0.0 -0.4074076...,System\n1.0\n2.61659 0.0 -0.03638\n-0.06603 3....


In [52]:
import pandas as pd
info = {}
test_path = pd.read_csv("/content/cdvae/data/carbon_24/test.csv")
val_path = pd.read_csv("/content/cdvae/data/carbon_24/val.csv")
train_path = pd.read_csv("/content/cdvae/data/carbon_24/train.csv")
test={}
val={}
train={}

for i,ii in train_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  train[jid]=pos

for i,ii in val_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  val[jid]=pos

for i,ii in test_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  test[jid]=pos
info['train']=train
info['val']=val
info['test']=test

In [55]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=info,filename='talhatest.json')

In [56]:
test_path

,Unnamed: 0,material_id,cif,energy_per_atom
0,56,C-13927-8536-14,# generated using pymatgen\ndata_C\n_symmetry_...,-154.546684
1,3900,C-72728-4135-43,# generated using pymatgen\ndata_C\n_symmetry_...,-154.159400
2,863,C-92152-5470-22,# generated using pymatgen\ndata_C\n_symmetry_...,-154.158839
3,4676,C-170380-2255-20,# generated using pymatgen\ndata_C\n_symmetry_...,-154.313358
4,1157,C-193944-7687-47,# generated using pymatgen\ndata_C\n_symmetry_...,-154.305662
...,...,...,...,...
94,3342,C-56503-8782-7,# generated using pymatgen\ndata_C\n_symmetry_...,-154.340324
95,5882,C-102903-5111-47,# generated using pymatgen\ndata_C\n_symmetry_...,-154.146013
96,7215,C-34617-8887-22,# generated using pymatgen\ndata_C\n_symmetry_...,-154.530711
97,2072,C-57148-8436-25,# generated using pymatgen\ndata_C\n_symmetry_...,-154.202201


In [59]:
!cp scripts/compute_metrics.py scripts/eval_utils.py .

In [60]:
!ls

AI-AtomGen-energy_per_atom-carbon24-test-rmse.csv  conf		eval_utils.py	  scripts
assets						   data		HYDRA_JOBS	  setup.py
cdvae						   env.cpu.yml	LICENSE		  talhatest.json
cdvae.egg-info					   env_sub.yml	README.md	  WABDB
compute_metrics.py				   env.yml	requirements.txt


In [61]:
from p_tqdm import p_map
from compute_metrics import GenEval,get_crystal_array_list,Crystal, RecEval
crys_array_list, true_crystal_array_list = get_crystal_array_list(recon_path)
pred_crys = p_map(lambda x: Crystal(x), crys_array_list)
gt_crys = p_map(lambda x: Crystal(x), true_crystal_array_list)
rec_evaluator = RecEval(pred_crys, gt_crys)
recon_metrics = rec_evaluator.get_metrics()


crys_array_list, _ = get_crystal_array_list(gen_path)
gen_crys = p_map(lambda x: Crystal(x), crys_array_list)
gen_evaluator = GenEval(gen_crys, gt_crys, eval_model_name='carbon')
gen_metrics = gen_evaluator.get_metrics()
print(recon_metrics)

/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")


  0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


  0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(
100%|██████████| 99/99 [00:01<00:00, 51.93it/s]


...

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/content/cdvae/cdvae/common/data_utils.py:622: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float)
/content/cdvae/eval_utils.py:56: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(model_path)):
/usr/local/lib/python3.10/dist-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
/content/cdvae/eval_utils.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a 

  0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'comp_valid': 1.0, 'struct_valid': 0.96, 'valid': 0.96, 'comp_div': nan, 'struct_div': nan, 'wdist_density': 0.3124391909415668, 'wdist_num_elems': 0.0, 'wdist_prop': 3.365540937943892}
{'match_rate': 0.15151515151515152, 'rms_dist': 0.3790775816257193}


/content/cdvae/cdvae/common/data_utils.py:622: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float)


In [62]:
print(recon_metrics)

{'match_rate': 0.15151515151515152, 'rms_dist': 0.3790775816257193}


In [63]:
print(gen_metrics)

{'comp_valid': 1.0, 'struct_valid': 0.96, 'valid': 0.96, 'comp_div': nan, 'struct_div': nan, 'wdist_density': 0.3124391909415668, 'wdist_num_elems': 0.0, 'wdist_prop': 3.365540937943892, 'cov_recall': 0.0, 'cov_precision': 0.0, 'amsd_recall': 0.8427693987987747, 'amsd_precision': 0.7298330763307811, 'amcd_recall': 0.0, 'amcd_precision': 0.0}


In [ ]:
!pip freeze